<a href="https://colab.research.google.com/github/ANMOLSHRI8980/Deep-Learning/blob/main/Pump_Maintenance_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np

def generate_dummy_data(n_samples=1000):
    np.random.seed(42)
    data = {
        'flow_rate': np.random.normal(100, 10, n_samples),  # m³/h (cubic meters per hour)
        'pressure': np.random.normal(50, 5, n_samples),  # bar
        'head': np.random.normal(30, 3, n_samples),  # m (meters)
        'power': np.random.normal(1000, 100, n_samples),  # kW (kilowatts)
        'speed': np.random.normal(1800, 50, n_samples),  # rpm (revolutions per minute)
        'npsh': np.random.normal(10, 1, n_samples),  # m (meters)
        'vibration_amplitude': np.random.normal(0.5, 0.1, n_samples),  # mm/s (millimeters per second)
        'vibration_frequency': np.random.normal(60, 5, n_samples),  # Hz (Hertz)
        'motor_temperature': np.random.normal(70, 5, n_samples),  # °C (degrees Celsius)
        'bearing_temperature': np.random.normal(60, 5, n_samples),  # °C (degrees Celsius)
        'inlet_pressure': np.random.normal(30, 3, n_samples),  # bar
        'outlet_pressure': np.random.normal(80, 5, n_samples),  # bar
        'water_velocity': np.random.normal(5, 0.5, n_samples),  # m/s (meters per second)
        'current_draw': np.random.normal(50, 5, n_samples),  # A (Amperes)
        'voltage': np.random.normal(220, 10, n_samples),  # V (Volts)
        'torque': np.random.normal(100, 10, n_samples),  # Nm (Newton meters)
        'runtime_hours': np.cumsum(np.random.uniform(0, 24, n_samples)),  # hours
        'ambient_temperature': np.random.normal(25, 3, n_samples),  # °C (degrees Celsius)
        'humidity': np.random.uniform(30, 70, n_samples),  # % (percentage)
        'noise_level': np.random.normal(70, 5, n_samples),  # dB (decibels)
        'bearing_frequency': np.random.normal(100, 10, n_samples),  # Hz (Hertz)
        'bearing_amplitude': np.random.normal(0.1, 0.02, n_samples),  # g (acceleration due to gravity)
        'start_stop_cycles': np.cumsum(np.random.randint(0, 2, n_samples))  # count (unitless)
    }

    df = pd.DataFrame(data)

    # failure events
    df['failure'] = 0
    failure_probability = 0.05
    df.loc[np.random.random(n_samples) < failure_probability, 'failure'] = 1

    # correlations between features and failures
    df.loc[df['failure'] == 1, 'vibration_amplitude'] *= 1.5
    df.loc[df['failure'] == 1, 'bearing_temperature'] *= 1.2
    df.loc[df['failure'] == 1, 'noise_level'] *= 1.3
    df.loc[df['failure'] == 1, 'bearing_frequency'] *= 1.4
    df.loc[df['failure'] == 1, 'bearing_amplitude'] *= 2.0

    return df

# Generating dummy data
data = generate_dummy_data(10000)

data.to_excel("enhanced_pump_maintenance_data_with_units.xlsx", index=False)

print("\nData statistics:")
print(data.describe())

print("\nFailure rate:")
print(data['failure'].mean())


Data statistics:
          flow_rate      pressure          head         power         speed  \
count  10000.000000  10000.000000  10000.000000  10000.000000  10000.000000   
mean      99.978640     50.067670     29.962612    999.242852   1800.326584   
std       10.034624      5.005051      2.974254    100.448707     50.019853   
min       60.775997     30.718123     19.034740    553.439614   1585.230451   
25%       93.274095     46.689945     27.899544    930.538715   1766.796613   
50%       99.974050     50.079234     29.982695    998.961775   1800.478034   
75%      106.710809     53.469324     31.991692   1067.903045   1834.191748   
max      139.262377     72.395421     41.074874   1372.783334   1976.826583   

               npsh  vibration_amplitude  vibration_frequency  \
count  10000.000000         10000.000000         10000.000000   
mean      10.001746             0.513806            59.993345   
std        1.009735             0.116331             5.034910   
min       

In [8]:
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, TimeDistributed
from tensorflow.keras.optimizers import Adam
from sklearn.impute import SimpleImputer
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import StackingClassifier
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.manifold import TSNE
import os

class EnhancedAdvancedHybridPumpMaintenanceModel:
    def __init__(self, sequence_length=50):
        self.sequence_length = sequence_length
        self.scaler = StandardScaler()
        self.imputer = SimpleImputer(strategy='mean')
        self.rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
        self.svm_model = SVC(probability=True, random_state=42, class_weight='balanced')
        self.xgb_model = XGBClassifier(random_state=42, scale_pos_weight=19)
        self.lstm_model = None
        self.autoencoder = None
        self.arima_models = {}
        self.stacking_model = None
        self.feature_selector = None

    def preprocess_data(self, X, y=None, is_training=True):
        X = self.imputer.fit_transform(X) if is_training else self.imputer.transform(X)
        X = self.scaler.fit_transform(X) if is_training else self.scaler.transform(X)
        if y is not None:
            return X, y
        return X

    def create_sequences(self, X, y=None):
        X_seq = []
        y_seq = []
        for i in range(len(X) - self.sequence_length + 1):
            X_seq.append(X[i:i+self.sequence_length])
            if y is not None:
                y_seq.append(y[i+self.sequence_length-1])
        return np.array(X_seq), np.array(y_seq) if y is not None else None

    def build_lstm_model(self, input_shape):
        model = Sequential([
            Bidirectional(LSTM(64, return_sequences=True, input_shape=input_shape)),
            Dropout(0.2),
            Bidirectional(LSTM(32, return_sequences=True)),
            Dropout(0.2),
            TimeDistributed(Dense(16, activation='relu')),
            LSTM(16),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
        return model

    def build_autoencoder(self, input_shape):
        model = Sequential([
            Dense(32, activation="relu", input_shape=input_shape),
            Dense(16, activation="relu"),
            Dense(8, activation="relu"),
            Dense(16, activation="relu"),
            Dense(32, activation="relu"),
            Dense(input_shape[0], activation="linear")
        ])
        model.compile(optimizer='adam', loss='mse')
        return model

    def fit(self, X, y):
        X_processed, y = self.preprocess_data(X, y)

        X_resampled, y_resampled = self._resample(X_processed, y)

        # Feature selection
        self.feature_selector = SelectFromModel(XGBClassifier(random_state=42))
        X_selected = self.feature_selector.fit_transform(X_resampled, y_resampled)

        X_train, X_val, y_train, y_val = train_test_split(X_selected, y_resampled, test_size=0.2, random_state=42)

        # Train traditional ML models
        self.rf_model.fit(X_train, y_train)
        self.svm_model.fit(X_train, y_train)
        self.xgb_model.fit(X_train, y_train)

        # Prepare sequences for LSTM
        X_seq, y_seq = self.create_sequences(X_selected, y_resampled)
        X_seq_train, X_seq_val, y_seq_train, y_seq_val = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

        # Train LSTM model
        self.lstm_model = self.build_lstm_model((X_seq.shape[1], X_seq.shape[2]))
        self.lstm_model.fit(X_seq_train, y_seq_train, validation_data=(X_seq_val, y_seq_val), epochs=5, batch_size=32, verbose=1)

        # Train autoencoder
        self.autoencoder = self.build_autoencoder((X_selected.shape[1],))
        self.autoencoder.fit(X_selected, X_selected, epochs=5, batch_size=32, validation_split=0.2, verbose=1)

        # Train ARIMA models for each feature
        for i in range(X_selected.shape[1]):
            try:
                model = ARIMA(X_selected[:, i], order=(1,1,1))
                self.arima_models[i] = model.fit()
            except:
                print(f"Unable to fit ARIMA model for feature {i}")

        # Train stacking ensemble
        base_models = [
            ('rf', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')),
            ('svm', SVC(probability=True, random_state=42, class_weight='balanced')),
            ('xgb', XGBClassifier(random_state=42, scale_pos_weight=19))
        ]
        self.stacking_model = StackingClassifier(estimators=base_models, final_estimator=RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
        self.stacking_model.fit(X_train, y_train)

    def visualize_results(self, X, y, predictions_1, predictions_2, predictions_3, anomaly_scores, arima_forecasts):
        plt.figure(figsize=(20, 30))

        # 1. Failure Probability (Different Approaches)
        plt.subplot(5, 2, 1)
        plt.plot(predictions_1, label='Approach 1')
        plt.plot(predictions_2, label='Approach 2')
        plt.plot(predictions_3, label='Approach 3')
        plt.title('Failure Probability (Different Approaches)')
        plt.legend()

        # 2. Anomaly Scores
        plt.subplot(5, 2, 2)
        plt.plot(anomaly_scores)
        plt.title('Anomaly Scores')

        # 3. ARIMA Forecast (First Feature)
        plt.subplot(5, 2, 3)
        plt.plot(arima_forecasts[:, 0])
        plt.title('ARIMA Forecast (First Feature)')

        # 4. Actual vs Predicted Failures
        plt.subplot(5, 2, 4)
        plt.plot(y, label='Actual')
        plt.plot((predictions_3 > 0.5).astype(int), label='Predicted (Approach 3)')
        plt.title('Actual vs Predicted Failures')
        plt.legend()

        # 5. Feature Importance
        feature_importance = self.rf_model.feature_importances_
        feature_names = X.columns[self.feature_selector.get_support()]
        importance_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importance})
        importance_df = importance_df.sort_values('importance', ascending=False)

        plt.subplot(5, 2, 5)
        sns.barplot(x='importance', y='feature', data=importance_df.head(10))
        plt.title('Top 10 Feature Importance')

        # 6. Confusion Matrix
        plt.subplot(5, 2, 6)
        cm = confusion_matrix(y[self.sequence_length-1:], (predictions_3[self.sequence_length-1:] > 0.5).astype(int))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')

        # 7. ROC Curve
        plt.subplot(5, 2, 7)
        fpr, tpr, _ = roc_curve(y[self.sequence_length-1:], predictions_3[self.sequence_length-1:])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")

        # 8. Anomaly Score Distribution
        plt.subplot(5, 2, 8)
        sns.histplot(anomaly_scores, kde=True)
        plt.title('Anomaly Score Distribution')

        # 9. Time Series of Key Features
        plt.subplot(5, 2, 9)
        for feature in importance_df['feature'].head(3):  # Plot top 3 important features
            plt.plot(X[feature], label=feature)
        plt.title('Time Series of Top 3 Important Features')
        plt.legend()

        # 10. t-SNE Visualization
        plt.subplot(5, 2, 10)
        tsne = TSNE(n_components=2, random_state=42)
        X_processed = self.preprocess_data(X, is_training=False)
        X_selected = self.feature_selector.transform(X_processed)
        X_tsne = tsne.fit_transform(X_selected)
        plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='viridis')
        plt.title('t-SNE Visualization of Features')
        plt.colorbar(label='Failure')

        plt.tight_layout()
        plt.show()

    def _resample(self, X, y):
        X_majority = X[y == 0]
        X_minority = X[y == 1]
        y_majority = y[y == 0]
        y_minority = y[y == 1]

        X_minority_upsampled, y_minority_upsampled = resample(
            X_minority, y_minority, replace=True, n_samples=len(X_majority), random_state=42
        )

        X_resampled = np.vstack((X_majority, X_minority_upsampled))
        y_resampled = np.hstack((y_majority, y_minority_upsampled))

        return X_resampled, y_resampled

    def predict(self, X):
        X_processed = self.preprocess_data(X, is_training=False)
        X_selected = self.feature_selector.transform(X_processed)

        rf_pred = self.rf_model.predict_proba(X_selected)[:, 1]
        svm_pred = self.svm_model.predict_proba(X_selected)[:, 1]
        xgb_pred = self.xgb_model.predict_proba(X_selected)[:, 1]

        X_seq, _ = self.create_sequences(X_selected)
        lstm_pred = self.lstm_model.predict(X_seq).flatten()

        stacking_pred = self.stacking_model.predict_proba(X_selected)[:, 1]

        reconstructed = self.autoencoder.predict(X_selected)
        mse = np.mean(np.power(X_selected - reconstructed, 2), axis=1)
        anomaly_scores = (mse - np.min(mse)) / (np.max(mse) - np.min(mse))

        arima_forecasts = np.zeros_like(X_selected)
        for i in range(X_selected.shape[1]):
            if i in self.arima_models:
                forecast = self.arima_models[i].forecast(steps=1)
                arima_forecasts[:, i] = forecast

        combined_pred_1 = self._combine_predictions_approach_1(rf_pred, svm_pred, xgb_pred, lstm_pred, stacking_pred, anomaly_scores)
        combined_pred_2 = self._combine_predictions_approach_2(rf_pred, svm_pred, xgb_pred, lstm_pred, stacking_pred, anomaly_scores)
        combined_pred_3 = self._combine_predictions_approach_3(rf_pred, svm_pred, xgb_pred, lstm_pred, stacking_pred, anomaly_scores)

        return combined_pred_1, combined_pred_2, combined_pred_3, anomaly_scores, arima_forecasts

    def _combine_predictions_approach_1(self, rf_pred, svm_pred, xgb_pred, lstm_pred, stacking_pred, anomaly_scores):
        n = len(rf_pred)
        m = len(lstm_pred)
        pad_length = n - m
        lstm_pred_padded = np.pad(lstm_pred, (0, pad_length), mode='edge')
        return (rf_pred + svm_pred + xgb_pred + lstm_pred_padded + stacking_pred + anomaly_scores) / 6

    def _combine_predictions_approach_2(self, rf_pred, svm_pred, xgb_pred, lstm_pred, stacking_pred, anomaly_scores):
        n = len(rf_pred)
        m = len(lstm_pred)
        combined_pred = np.zeros(n)
        combined_pred[:m] = (rf_pred[:m] + svm_pred[:m] + xgb_pred[:m] + lstm_pred + stacking_pred[:m] + anomaly_scores[:m]) / 6
        combined_pred[m:] = (rf_pred[m:] + svm_pred[m:] + xgb_pred[m:] + stacking_pred[m:] + anomaly_scores[m:]) / 5
        return combined_pred

    def _combine_predictions_approach_3(self, rf_pred, svm_pred, xgb_pred, lstm_pred, stacking_pred, anomaly_scores):
        n = len(rf_pred)
        m = len(lstm_pred)
        pad_length = n - m
        lstm_pred_padded = np.pad(lstm_pred, (0, pad_length), mode='edge')
        weights = np.array([0.2, 0.15, 0.2, 0.25, 0.15, 0.05])
        return np.average(np.vstack((rf_pred, svm_pred, xgb_pred, lstm_pred_padded, stacking_pred, anomaly_scores)), axis=0, weights=weights)

def generate_dummy_data(n_samples=1000):
    np.random.seed(42)
    vibration = np.random.normal(0, 1, n_samples)
    temperature = np.random.normal(60, 10, n_samples)
    pressure = np.random.normal(100, 15,n_samples)